In [32]:
from JSU_lib import *

## Getting rep_chain sequenecs

In [33]:
aa_code = {
    'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D',
    'CYS': 'C', 'GLU': 'E', 'GLN': 'Q', 'GLY': 'G',
    'HIS': 'H', 'ILE': 'I', 'LEU': 'L', 'LYS': 'K',
    'MET': 'M', 'PHE': 'F', 'PRO': 'P', 'SER': 'S',
    'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V',
    'SEC': 'U', 'PYL': 'O', 'UNK': 'X'
}


In [34]:
PUResNet_dir = "./../DATA/PUResNet_SERVER"

In [35]:
rep_chains_dir = './../DATA/clean_rep_chains'

In [36]:
LIGYSIS_reps = read_from_pickle("./results/LIGYSIS_3448_chains.pkl")

In [37]:
clean_files = sorted([el for el in os.listdir(rep_chains_dir) if el.split(".")[0] in LIGYSIS_reps])

print(len(clean_files))

3448


In [7]:
seq_dict = {}

for i, clean_file in enumerate(clean_files):
    if i % 100 == 0:
        print(i)
    rep_chain = clean_file.split(".")[0]
    pdb_path = os.path.join(rep_chains_dir, clean_file)
    df = PDBXreader(inputfile=pdb_path).atoms(format_type="pdb", excluded=()).query('label_atom_id == "CA"').reset_index(drop = True)
    aas = df.label_comp_id.tolist()
    seq = "".join([aa_code[aa] for aa in aas])
    seq_dict[rep_chain] = seq

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400


In [8]:
len(seq_dict)

3448

In [9]:
save_to_pickle(seq_dict, "./results/LIGYSIS_3448_seqs_dict.pkl")

## Matching PUResNet folders to rep_chain

In [11]:
no_prot = []
unmapped = []

PURESNET_dirs = os.listdir(PUResNet_dir)

mapping_dict = {}

for i, PURESNET_job in enumerate(PURESNET_dirs):
    
    if i % 100 == 0:
        print(i)

    # if i > 9:
    #     break
    
    mapping_dict[PURESNET_job] = []

    pdb_path = os.path.join(PUResNet_dir, PURESNET_job, 'pdb', 'protein.pdb')

    try:
        df = PDBXreader(inputfile=pdb_path).atoms(format_type="pdb", excluded=()).query('label_atom_id == "CA"').reset_index(drop = True)
        aas = df.label_comp_id.tolist()
        query_seq = "".join([aa_code[aa] for aa in aas])
        for rep_chain, target_seq in seq_dict.items():
            if query_seq == target_seq:
                mapping_dict[PURESNET_job].append(rep_chain)
        if mapping_dict[PURESNET_job] != []:
            print(f'Mapped {PURESNET_job} to {mapping_dict[PURESNET_job]}')
        else:
            print(f'Failed to map {PURESNET_job}')
            unmapped.append(PURESNET_job)
            
    except:
        print('ERROR: protein file not found')
        no_prot.append(PURESNET_job)
    #break

0
Mapped 2024-05-21_11-39-32.667 to ['7qwq_v']
Mapped 2024-05-21_11-18-37.705 to ['7k36_I']
Mapped 2024-05-21_07-11-30.239 to ['2vq3_A']
Mapped 2024-05-20_19-57-57.588 to ['1szb_B']
Mapped 2024-05-21_09-21-52.765 to ['5kwy_D']
Mapped 2024-05-21_08-34-23.789 to ['4l1j_B']
Mapped 2024-05-21_10-41-37.59 to ['6s8l_A']
Mapped 2024-05-21_06-19-52.876 to ['2em9_A']
Mapped 2024-05-21_09-01-42.658 to ['5a9j_D']
Mapped 2024-05-21_10-22-37.854 to ['6m02_B']
Mapped 2024-05-21_07-01-05.681 to ['2pl0_A']
Mapped 2024-05-21_09-29-09.551 to ['5nfo_B']
Mapped 2024-05-21_12-11-02.913 to ['8cv5_A']
Mapped 2024-05-21_11-38-49.979 to ['7qqh_D']
Mapped 2024-05-20_16-58-59.876 to ['1gxd_A']
Mapped 2024-05-21_05-57-58.632 to ['1yb5_A']
Mapped 2024-05-21_11-53-00.16 to ['7vox_H']
Mapped 2024-05-21_06-15-37.459 to ['2e5r_A']
Mapped 2024-05-21_06-36-11.541 to ['2hrb_A']
Mapped 2024-05-21_10-36-35.138 to ['6qav_C']
Mapped 2024-05-21_07-59-35.401 to ['3q2g_B']
Mapped 2024-05-21_10-11-33.84 to ['6hts_R']
Mapped 2024

In [13]:
print(len(no_prot))

print(len(unmapped))

print(len([k for k, v in mapping_dict.items() if v != []]))

17
15
3457


In [15]:
print(len([k for k, v in mapping_dict.items() if len(v) > 1]))

0


In [17]:
correct_jobs = []
for k, v in mapping_dict.items():
    if len(v) == 1:
        correct_jobs.append(v[0])
        
print(len(correct_jobs))

3457


In [18]:
missing = [el for el in LIGYSIS_reps if el not in correct_jobs]

print(len(missing))

print(missing)

0
[]


In [20]:
save_to_pickle(mapping_dict, "./results/PURESNET_jobs_mapping.pkl")

In [38]:
mapping_dict = read_from_pickle("./results/PURESNET_jobs_mapping.pkl")

In [39]:
rep2job = {}
for job_id, rep_chains in mapping_dict.items():
    if rep_chains == []:
        continue
    rep_chain = rep_chains[0]
    if rep_chain not in rep2job:
        rep2job[rep_chain] = []
    rep2job[rep_chain].append(job_id)

In [40]:
len(rep2job)

3448

In [41]:
for k, v in rep2job.items():
    if len(v) > 1:
        print(k, v)

1a52_A ['2024-05-20_15-39-41.934', '2024-05-20_12-39-03.998', '2024-05-20_16-00-23.708', '2024-05-20_15-45-40.222', '2024-05-20_14-21-33.557', '2024-05-20_14-29-37.464', '2024-05-20_14-12-45.25']
1b1c_A ['2024-05-20_14-24-59.94', '2024-05-20_16-36-10.648']
1wlj_A ['2024-05-21_05-52-56.167', '2024-05-20_21-28-44.442']
1blx_A ['2024-05-20_16-49-03.585', '2024-05-20_16-06-14.248']


In [42]:
for k, v in rep2job.items():
    if len(v) > 1:
        print(k, sorted(v, reverse = True)) # keeping latest one is the final run on clean files

1a52_A ['2024-05-20_16-00-23.708', '2024-05-20_15-45-40.222', '2024-05-20_15-39-41.934', '2024-05-20_14-29-37.464', '2024-05-20_14-21-33.557', '2024-05-20_14-12-45.25', '2024-05-20_12-39-03.998']
1b1c_A ['2024-05-20_16-36-10.648', '2024-05-20_14-24-59.94']
1wlj_A ['2024-05-21_05-52-56.167', '2024-05-20_21-28-44.442']
1blx_A ['2024-05-20_16-49-03.585', '2024-05-20_16-06-14.248']


In [44]:
rep2job_filt = {k: sorted(v, reverse = True)[0] for k, v in rep2job.items()}

In [45]:
rep2job_filt_sort = {k: rep2job_filt[k] for k in LIGYSIS_reps}

## Extract pocket information

In [46]:
pocket_files = {}
for i, rep_chain in enumerate(rep2job_filt_sort.keys()):
    if i % 100 == 0:
        print(i)
    job_id = rep2job_filt_sort[rep_chain]
    pocket_dir = os.path.join(PUResNet_dir, job_id, 'pocket')
    pocket_files[rep_chain] = sorted(os.listdir(pocket_dir))
    #break

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400


In [48]:
pocket_files_filt = {}
without_pocket_files = {}
for k, v in pocket_files.items():
    if v == []:
        continue
    else:
        pocket_files_filt[k] = []
        without_pocket_files[k] = []
        for el in v:
            if "without" in el:
                without_pocket_files[k].append(el)
            else:
                pocket_files_filt[k].append(el)

In [49]:
print(len(pocket_files))
print(len(pocket_files_filt))
print(len(without_pocket_files))

3448
3146
3146


In [47]:
3448 - 3146 # proteins chains where PUResNet does not predict anything

302

In [50]:
set([len(v) for v in without_pocket_files.values()]) # all 3,146 chains with pocket predictions have 1 "without_clus_pkt" file

{1}

In [51]:
pocket_files["8jhf_A"]

['without_clus_pkt.pdb']

In [52]:
pocket_files_filt2 = {}
for k, v in pocket_files_filt.items():
    if len(v) == 0: # these are protein chains for which residues are predicted, but they don't cluster together. Perhaps singletons
        continue
    else:
        pocket_files_filt2[k] = v

In [53]:
3448 - len(pocket_files_filt2) #580 protein chains in total with no predicted pockets

580

In [54]:
n_pockets = [len(v) for v in pocket_files_filt2.values()]
n_pockets_un = set(n_pockets)
print(n_pockets_un)

{1, 2, 3, 4}


In [55]:
for el in n_pockets_un:
    count = n_pockets.count(el)
    print(el, count, round(100*count/len(n_pockets), 2))

1 2584 90.1
2 259 9.03
3 22 0.77
4 3 0.1


In [57]:
pocket_id_dict = {}
for i, rep_chain in enumerate(pocket_files_filt2.keys()):
    if i % 100 == 0:
        print(i)
    job_id = rep2job_filt_sort[rep_chain]
    pocket_files = pocket_files_filt2[rep_chain]
    for i, pocket_file in enumerate(pocket_files):
        pocket_name = pocket_file.split(".")[0]
        pocket_id_dict[(rep_chain, pocket_name)] = i+1
#
        #pocket_id_dict[]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800


In [59]:
save_to_pickle(pocket_id_dict, "./results/puresnet_pocket_name_dict.pkl")

In [86]:
rows = []
for i, rep_chain in enumerate(pocket_files_filt2.keys()):
    if i % 100 == 0:
        print(i)
    job_id = rep2job_filt_sort[rep_chain]
    pocket_files = pocket_files_filt2[rep_chain]
    for i, pocket_file in enumerate(pocket_files):
        pocket_file_path = os.path.join(PUResNet_dir, job_id, "pocket", pocket_file)
        pocket_df = PDBXreader(inputfile=pocket_file_path).atoms(format_type="pdb", excluded=())
        pocket_coords = np.array(list((zip(pocket_df.Cartn_x, pocket_df.Cartn_y, pocket_df.Cartn_z))))
        centroid = np.mean(pocket_coords, axis=0)
        centroid = tuple([round(el, 4) for el in centroid])
        ress_CAs = pocket_df.query('label_atom_id == "CA"')
        ress = ress_CAs.label_seq_id_full.tolist()
        CAs_coords = np.array(list((zip(ress_CAs.Cartn_x, ress_CAs.Cartn_y, ress_CAs.Cartn_z))))
        center_of_mass = np.mean(CAs_coords, axis=0)
        distances_squared = np.sum((CAs_coords - center_of_mass)**2, axis=1)
        radius_of_gyration = round(np.sqrt(np.mean(distances_squared)), 2)
        d_row = [rep_chain, i+1, len(ress_CAs), radius_of_gyration, centroid, ress]
        rows.append(d_row)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800


In [88]:
PURESNET_pockets = pd.DataFrame(rows, columns = ["rep_chain", "ID", "n_aas", "RoG", "centre", "aas"])

In [89]:
PURESNET_pockets

,rep_chain,ID,n_aas,RoG,centre,aas
0,1a52_A,1,22,8.81,"(106.4784, 15.7626, 96.0529)","[47, 50, 51, 53, 54, 57, 87, 88, 91, 92, 95, 9..."
1,1a5h_B,1,24,8.34,"(17.5409, 53.2903, 27.6191)","[47, 90, 91, 92, 93, 174, 197, 198, 199, 200, ..."
2,1a9w_E,1,21,9.56,"(3.6129, 33.9262, 47.0811)","[31, 38, 41, 42, 45, 63, 66, 67, 70, 71, 85, 8..."
3,1au1_A,1,6,5.56,"(2.9895, 6.9643, 31.9137)","[27, 28, 32, 144, 145, 148]"
4,1aue_B,1,7,5.99,"(27.6331, 11.3084, 20.9968)","[17, 18, 21, 25, 87, 91, 94]"
...,...,...,...,...,...,...
3175,8y14_A,1,8,6.63,"(-31.5191, 2.0611, 8.443)","[149, 150, 152, 185, 186, 189, 192, 194]"
3176,8y14_A,2,5,5.22,"(-22.7398, -20.0204, 33.0662)","[292, 293, 332, 333, 337]"
3177,8y14_A,3,2,3.14,"(-9.1276, -9.4995, 44.9326)","[367, 425]"
3178,8y14_A,4,1,0.00,"(-2.1078, -29.6716, 26.4812)",[352]


## Mapping PDB residues to UP residues

In [90]:
up_aas = {}

errors = []

for _, row in PURESNET_pockets.iterrows():
    b = 0
    rep_chain = row.rep_chain
    site_ress = row.aas
    ID = row["ID"]
    mapping_dict = read_from_pickle(f'./../DATA/mappings_label_full/{rep_chain}.pkl')
    site_up_aas = []
    for res in site_ress:
        try:
            site_up_aas.append(mapping_dict[str(res)]) # I think there are integers here as it is not working with AltLocs, so none are present
        except:
            errors.append(rep_chain)
            b = 1
            break
    if b ==1:
        continue
    up_aas[(rep_chain, ID)] = site_up_aas

In [93]:
print(len(up_aas)) #3130
print(len(errors)) #50

3130
50


In [95]:
errors = list(set(errors)) # 49 chains have issues with SIFTS mapping
print(len(errors)) #49

49


In [96]:
save_to_pickle(errors, "./results/PURESNET_no_sifts.pkl")

In [97]:
PURESNET_pockets_w_sifts = PURESNET_pockets.query('rep_chain not in @errors').copy().reset_index(drop = True)
PURESNET_pockets_w_sifts["up_aas"] = PURESNET_pockets_w_sifts.set_index(['rep_chain', 'ID']).index.map(up_aas)

In [99]:
PURESNET_pockets_w_sifts["n_up_aas"] = PURESNET_pockets_w_sifts['up_aas'].apply(lambda x: len(x))
assert PURESNET_pockets_w_sifts.n_aas.equals(PURESNET_pockets_w_sifts.n_up_aas)
PURESNET_pockets_w_sifts.drop(columns = ["n_up_aas"], inplace = True)

In [100]:
PURESNET_pockets_w_sifts

,rep_chain,ID,n_aas,RoG,centre,aas,up_aas
0,1a52_A,1,22,8.81,"(106.4784, 15.7626, 96.0529)","[47, 50, 51, 53, 54, 57, 87, 88, 91, 92, 95, 9...","[343, 346, 347, 349, 350, 353, 383, 384, 387, ..."
1,1a5h_B,1,24,8.34,"(17.5409, 53.2903, 27.6191)","[47, 90, 91, 92, 93, 174, 197, 198, 199, 200, ...","[357, 400, 401, 402, 403, 484, 507, 508, 509, ..."
2,1a9w_E,1,21,9.56,"(3.6129, 33.9262, 47.0811)","[31, 38, 41, 42, 45, 63, 66, 67, 70, 71, 85, 8...","[32, 39, 42, 43, 46, 64, 67, 68, 71, 72, 86, 8..."
3,1au1_A,1,6,5.56,"(2.9895, 6.9643, 31.9137)","[27, 28, 32, 144, 145, 148]","[48, 49, 53, 165, 166, 169]"
4,1aue_B,1,7,5.99,"(27.6331, 11.3084, 20.9968)","[17, 18, 21, 25, 87, 91, 94]","[2031, 2032, 2035, 2039, 2101, 2105, 2108]"
...,...,...,...,...,...,...,...
3121,8y14_A,1,8,6.63,"(-31.5191, 2.0611, 8.443)","[149, 150, 152, 185, 186, 189, 192, 194]","[286, 287, 289, 322, 323, 326, 329, 331]"
3122,8y14_A,2,5,5.22,"(-22.7398, -20.0204, 33.0662)","[292, 293, 332, 333, 337]","[429, 430, 469, 470, 474]"
3123,8y14_A,3,2,3.14,"(-9.1276, -9.4995, 44.9326)","[367, 425]","[504, 562]"
3124,8y14_A,4,1,0.00,"(-2.1078, -29.6716, 26.4812)",[352],[489]


## Add pocket surfaces and volumes

In [102]:
master_SASA_dict = read_from_pickle("./results/master_SASA_dict.pkl")

In [103]:
PURESNET_pockets_w_sifts_SASA = calculate_total_sasa(PURESNET_pockets_w_sifts, master_SASA_dict)

In [3]:
PURESNET_pockets_w_sifts_SASA = pd.read_pickle("./results/PURESNET_pockets_DEF_TRANS.pkl")

In [4]:
vols_dict = read_from_pickle("./results/PURESNET_volumes_dict.pkl")
PURESNET_pockets_w_sifts_SASA["VOL"] = PURESNET_pockets_w_sifts_SASA.set_index(['rep_chain', 'ID']).index.map(vols_dict)

In [105]:
PURESNET_pockets_w_sifts_SASA['centre_mat'] = PURESNET_pockets_w_sifts_SASA.apply(apply_rotation, axis=1)

In [8]:
PURESNET_pockets_w_sifts_SASA

,rep_chain,ID,n_aas,RoG,centre,aas,up_aas,SASA,centre_mat,VOL
0,1a52_A,1,22,8.81,"(106.4784, 15.7626, 96.0529)","[47, 50, 51, 53, 54, 57, 87, 88, 91, 92, 95, 9...","[343, 346, 347, 349, 350, 353, 383, 384, 387, ...",572.40,"(1.459, -8.69, -1.716)",506.0
1,1a5h_B,1,24,8.34,"(17.5409, 53.2903, 27.6191)","[47, 90, 91, 92, 93, 174, 197, 198, 199, 200, ...","[357, 400, 401, 402, 403, 484, 507, 508, 509, ...",790.91,"(-0.296, 7.11, -7.752)",725.0
2,1a9w_E,1,21,9.56,"(3.6129, 33.9262, 47.0811)","[31, 38, 41, 42, 45, 63, 66, 67, 70, 71, 85, 8...","[32, 39, 42, 43, 46, 64, 67, 68, 71, 72, 86, 8...",857.24,"(-4.619, 5.824, 1.578)",886.0
3,1au1_A,1,6,5.56,"(2.9895, 6.9643, 31.9137)","[27, 28, 32, 144, 145, 148]","[48, 49, 53, 165, 166, 169]",417.02,"(1.298, 17.464, -6.397)",132.0
4,1aue_B,1,7,5.99,"(27.6331, 11.3084, 20.9968)","[17, 18, 21, 25, 87, 91, 94]","[2031, 2032, 2035, 2039, 2101, 2105, 2108]",397.87,"(6.466, 3.551, 3.187)",92.0
...,...,...,...,...,...,...,...,...,...,...
3121,8y14_A,1,8,6.63,"(-31.5191, 2.0611, 8.443)","[149, 150, 152, 185, 186, 189, 192, 194]","[286, 287, 289, 322, 323, 326, 329, 331]",554.97,"(-6.181, -15.776, -12.236)",108.0
3122,8y14_A,2,5,5.22,"(-22.7398, -20.0204, 33.0662)","[292, 293, 332, 333, 337]","[429, 430, 469, 470, 474]",385.74,"(-1.971, 17.702, -6.533)",35.0
3123,8y14_A,3,2,3.14,"(-9.1276, -9.4995, 44.9326)","[367, 425]","[504, 562]",138.00,"(7.138, 18.715, 12.249)",0.0
3124,8y14_A,4,1,0.00,"(-2.1078, -29.6716, 26.4812)",[352],[489],174.85,"(-20.016, 24.559, 7.235)",0.0


In [7]:
PURESNET_pockets_w_sifts_SASA.to_pickle("./results/PURESNET_pockets_DEF_TRANS.pkl")

## Generating .ds file for PRANK

In [19]:
len(rep2job_filt_sort)

3448

In [21]:
rep2job_filt_sort["1a52_A"]

'2024-05-20_16-00-23.708'

In [26]:
for rep_chain, job_id in rep2job_filt_sort.items():
    #rep_chain = file.split(".")[0]
    protein_pdb = f'./../DATA/PUResNet_SERVER/{job_id}/pdb/protein.pdb'
    renamed_pdb = f'./../DATA/PUResNet_SERVER/{job_id}/pdb/{rep_chain}.pdb'
    shutil.copy(protein_pdb, renamed_pdb)
    print(f'./../../DATA/PUResNet_SERVER/{job_id}/pdb/{rep_chain}.pdb ./../../DATA/PUResNet_SERVER/{job_id}/pocket')
    #break

# running PRANK on it took 1 minutes 21.719 seconds with -threads 8

./../../DATA/PUResNet_SERVER/2024-05-20_16-00-23.708/pdb/1a52_A.pdb ./../../DATA/PUResNet_SERVER/2024-05-20_16-00-23.708/pocket
./../../DATA/PUResNet_SERVER/2024-05-20_16-33-30.833/pdb/1a5h_B.pdb ./../../DATA/PUResNet_SERVER/2024-05-20_16-33-30.833/pocket
./../../DATA/PUResNet_SERVER/2024-05-20_16-34-55.696/pdb/1a9w_E.pdb ./../../DATA/PUResNet_SERVER/2024-05-20_16-34-55.696/pocket
./../../DATA/PUResNet_SERVER/2024-05-20_16-35-07.654/pdb/1ags_B.pdb ./../../DATA/PUResNet_SERVER/2024-05-20_16-35-07.654/pocket
./../../DATA/PUResNet_SERVER/2024-05-20_16-35-20.26/pdb/1aii_A.pdb ./../../DATA/PUResNet_SERVER/2024-05-20_16-35-20.26/pocket
./../../DATA/PUResNet_SERVER/2024-05-20_16-35-32.341/pdb/1au1_A.pdb ./../../DATA/PUResNet_SERVER/2024-05-20_16-35-32.341/pocket
./../../DATA/PUResNet_SERVER/2024-05-20_16-35-44.907/pdb/1aue_B.pdb ./../../DATA/PUResNet_SERVER/2024-05-20_16-35-44.907/pocket
./../../DATA/PUResNet_SERVER/2024-05-20_16-35-57.156/pdb/1aye_A.pdb ./../../DATA/PUResNet_SERVER/2024-05-2

In [23]:
header = """
# Dataset for rescoring PUResNet predictions

PARAM.PREDICTION_METHOD=puresnet

HEADER: protein prediction

"""
for rep_chain, job_id in rep2job_filt_sort.items():
    #rep_chain = file.split(".")[0]
    print(f'./../../DATA/PUResNet_SERVER/{job_id}/pdb/protein.pdb ./../../DATA/PUResNet_SERVER/{job_id}/pocket')

# running PRANK on it took 1 minutes 21.719 seconds with -threads 8

./../../DATA/PUResNet_SERVER/2024-05-20_16-00-23.708/pdb/protein.pdb ./../../DATA/PUResNet_SERVER/2024-05-20_16-00-23.708/pocket
./../../DATA/PUResNet_SERVER/2024-05-20_16-33-30.833/pdb/protein.pdb ./../../DATA/PUResNet_SERVER/2024-05-20_16-33-30.833/pocket
./../../DATA/PUResNet_SERVER/2024-05-20_16-34-55.696/pdb/protein.pdb ./../../DATA/PUResNet_SERVER/2024-05-20_16-34-55.696/pocket
./../../DATA/PUResNet_SERVER/2024-05-20_16-35-07.654/pdb/protein.pdb ./../../DATA/PUResNet_SERVER/2024-05-20_16-35-07.654/pocket
./../../DATA/PUResNet_SERVER/2024-05-20_16-35-20.26/pdb/protein.pdb ./../../DATA/PUResNet_SERVER/2024-05-20_16-35-20.26/pocket
./../../DATA/PUResNet_SERVER/2024-05-20_16-35-32.341/pdb/protein.pdb ./../../DATA/PUResNet_SERVER/2024-05-20_16-35-32.341/pocket
./../../DATA/PUResNet_SERVER/2024-05-20_16-35-44.907/pdb/protein.pdb ./../../DATA/PUResNet_SERVER/2024-05-20_16-35-44.907/pocket
./../../DATA/PUResNet_SERVER/2024-05-20_16-35-57.156/pdb/protein.pdb ./../../DATA/PUResNet_SERVER/2

In [27]:
errors = [
    "1aye_A.pdb",
    "1kwm_A.pdb",
    "1op8_D.pdb",
    "2nna_A.pdb",
    "2zch_P.pdb",
    "3fvf_B.pdb",
    "4jze_H.pdb",
    "4wwy_B.pdb",
    "6n35_M.pdb",
    "6nvb_B.pdb",
    "6sma_B.pdb",
]

In [30]:
for error in errors:
    rep_chain = error.split(".")[0]
    print(rep2job_filt_sort[rep_chain])

2024-05-20_16-35-57.156
2024-05-20_17-05-51.198
2024-05-20_18-48-11.982
2024-05-21_06-55-51.791
2024-05-21_07-23-31.807
2024-05-21_07-39-03.653
2024-05-21_08-32-44.322
2024-05-21_08-52-29.985
2024-05-21_10-26-11.72
2024-05-21_10-28-28.738
2024-05-21_10-42-21.419
